In [2]:
%pip install -U langchain langchain_openai PyPDF2 faiss-cpu --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.93 requires langchain-openai<0.0.6,>=0.0.5, but you have langchain-openai 0.1.1 which is incompatible.
crewai 0.19.0 requires langchain-openai<0.0.6,>=0.0.5, but you have langchain-openai 0.1.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
from IPython.display import display, Markdown
from PyPDF2 import PdfReader
from langchain_openai import AzureOpenAIEmbeddings, AzureOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import LLMChain
import pandas as pd
from tqdm import tqdm

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
pdfreader = PdfReader('../data/Resume.pdf')
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

# Show first 1000 characters of the text.
raw_text[:1000]

'DI V Y A  PA R A M E S V A R A I Y E R  \n                                            Cell: 732-824-2905  \n                               Email: divsp6@gmail .com    \n \nPage 1 of 4  \nDynamic, proactive, and highly self-motivated  professional with 20+ years of people/ program  management \nexperience and  over 10 years of proven experience in the financial services industry. Exceedingly business -\nminded, results -oriented individual with exemplary capacity for  managing complex strategic  business \ntransformation projects with matrixed, cross -functional teams; and directing large global initiatives (over $1M \nand 10 months) resulting in widespread measurable ROI efficiencies.  I am PMP certified,  Certified Scrum \nMaster , have a BS in Computer Science and MBA in Global Management.  \n \n \nCORE COMPETENCIES  \n✓ Proficient abi lity to drive Business Transformations by laying out roadmap & managing execution  \n✓ Highly Detail Oriented with focus on balancing customer satisf

In [ ]:
OPENAI_DEPLOYMENT_NAME = "gpt-35-turbo"
OPENAI_MODEL_NAME = "gpt-35-turbo"
EMBEDDINGS_DEPLOYMENT_NAME = "text-embedding-ada-002"

llm = AzureOpenAI(azure_deployment=OPENAI_DEPLOYMENT_NAME, 
                  model_name=OPENAI_MODEL_NAME, 
                  openai_api_version=os.getenv("OPENAI_API_VERSION"),
                  temperature=0)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDINGS_DEPLOYMENT_NAME,
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
)

def ask_openai(question):
    llm_chain = LLMChain(prompt=question, llm=llm)
    return llm_chain.run(question)

def analyze_str(resume, options):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=800,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(resume)
    knowledge_base = FAISS.from_texts(chunks, embeddings)
    df_data = [{'option': option, 'value': []} for option in options]
    print("Fetching information")

    for i, option in tqdm(enumerate(options), desc="Fetching information", unit="option", ncols=100):
        question = f"What is this candidate's {option}? Please return the answer in a concise manner, no more than 250 words. If not found, return 'Not provided'"
        docs = knowledge_base.similarity_search(question)
        chain = load_qa_chain(llm, chain_type="stuff")
        response = chain.run(input_documents=docs, question=question )
        df_data[i]['value'] = response
        print("Looking for information: {option}")


    df = pd.DataFrame(df_data)
    print("Resume elements retrieved")
    return df

def analyze_resume(job_desc, resume, options):
    df = analyze_str(resume, options)
    df_string = df.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x).to_string(index=False)
    print("Analyzing with OpenAI..")
    summary_question = f"Job requirements: {{{job_desc}}}" + f"Resume summary: {{{df_string}}}" + "Please return a summary of the candidate's suitability for this position (limited to 200 words);'"
    summary = ask_openai(summary_question)
    df.loc[len(df)] = ['Summary', summary]
    extra_info = "Scoring criteria: Top 10 domestic universities +3 points, 985 universities +2 points, 211 universities +1 point, leading company experience +2 points, well-known company +1 point, overseas background +3 points, foreign company background +1 point."
    score_question = f"Job requirements: {{{job_desc}}}" + f"Resume summary: {{{df.to_string(index=False)}}}" + "Please return a matching score (0-100) for the candidate for this job, please score accurately to facilitate comparison with other candidates, '" + extra_info
    score = ask_openai(score_question)
    df.loc[len(df)] = ['Match Score', score]

    return df